In [12]:
import numpy as np
import xarray as xr
import rasterio as rio
import rioxarray

## read in data

In [2]:
data_path = '/mnt/Backups/gbrench/repos/rg_uavsar/data'

path_dic = {'swatch_00540':f'{data_path}/swatch_00540/geo',
            'swatch_08301':f'{data_path}/swatch_08301/geo',
            'swatch_09510':f'{data_path}/swatch_09510/geo',
            'swatch_17304_s1':f'{data_path}/swatch_17304_s1/geo',
            'swatch_17304_s2':f'{data_path}/swatch_17304_s2/geo',
            'swatch_18525':f'{data_path}/swatch_18525/geo',
            'swatch_26300':f'{data_path}/swatch_26300/geo',
            'swatch_27518':f'{data_path}/swatch_27518/geo'}

In [ ]:
# function to rewrite coordinates from metadata
def coord_range(ds):
    latrange = np.linspace(float(ds.attrs['Y_FIRST']),
                           ((float(ds.attrs['Y_STEP'])*float(ds.attrs['LENGTH']))+float(ds.attrs['Y_FIRST'])),
                           int(ds.attrs['LENGTH']))
    lonrange = np.linspace(float(ds.attrs['X_FIRST']),
                           ((float(ds.attrs['X_STEP'])*float(ds.attrs['WIDTH']))+float(ds.attrs['X_FIRST'])),
                           int(ds.attrs['WIDTH']))
    return latrange, lonrange

def mintpy2d_to_xarray(fn, crs):
    ds = xr.open_dataset(fn, engine='h5netcdf', phony_dims='sort')
    ds = ds.rename_dims({'phony_dim_0':'y',
                         'phony_dim_1':'x'
                        })
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'y': ('y', latrange),
                          'x': ('x', lonrange)})
    ds = ds.rio.write_crs(crs)
    
    return ds

In [ ]:
def open_uavsar(swatch, path):
    igram_ds = xr.open_dataset(f'{path}/{swatch}_20230725_20230925_unw_phase_m.tif').squeeze()
    cor_ds = xr.open_dataset(f'{path}/{swatch}_20230725_20230925_cor.tif').squeeze()
    geom_ds = mintpy2d_to_xarray(f'{path}/geo_geometryRadar.h5', crs='4326')
    geom_ds['cor'] = (('y', 'x'), cor_ds.band_data.values)
    geom_ds['displacement'] = (('y', 'x'), igram_ds.band_data.values)
    # geom_ds.rio.write_nodata(np.nan, inplace=True)
    # reproject to utm
    geom_ds = geom_ds.rio.reproject('EPSG:32613', nodata=np.nan)
    
    return geom_ds

In [10]:
for key, item in path_dic.items():
    print(key, item)

swatch_00540 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_00540
swatch_08301 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_08301
swatch_09510 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_09510
swatch_17304_s1 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s1
swatch_17304_s2 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2
swatch_18525 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_18525
swatch_26300 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_26300
swatch_27518 /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_27518
